<a href="https://colab.research.google.com/github/SolutionLr/DL/blob/main/pizza_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# import necessary modules 
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.datasets
import torchvision.transforms as transforms
from torch import nn
from sklearn.model_selection import train_test_split
from torchvision.models import vgg19
torch.manual_seed(0)



#Create Model
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.vgg = vgg19(pretrained=True)
        for param in self.vgg.parameters():
            param.requires_grad = False
        self.vgg.classifier[6] = nn.Sequential(nn.Linear(4096, 1), nn.Sigmoid())

    def forward(self, X):
        pred = self.vgg(X)
        return pred


def show_predict(model, loader):
    batch = next(iter(loader))
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)
    labels = labels.unsqueeze(1)
    labels = labels.to(torch.float32)
    pred = model(images)
    images = images.cpu()
    mean = torch.tensor([0.5539, 0.4266, 0.3235])
    mean = mean.reshape(1, 3, 1, 1)
    std = torch.tensor([0.3754, 0.2501, 0.1747])
    std = std.reshape(1, 3, 1, 1)
    images = (images * std) + mean
    grid = torchvision.utils.make_grid(images[0:8], nrow=4)
    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    print((pred[0:8] > 0.5).float().reshape(1, 8))
    plt.show()
    plt.close()


if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = VGG()
    model.to(device)

# Load and transform data
    path = '/content/drive/MyDrive/DL_1/pizza_not_classifier/pizza_dataset'
    transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5539, 0.4266, 0.3235], [0.3754, 0.2501, 0.1747])
                                    ])
    pizza_not_dataset = torchvision.datasets.ImageFolder(root=path, transform=transform)
    train_ind, test_ind = train_test_split(list(range(len(pizza_not_dataset.targets))), shuffle=True, test_size=0.2,
                                           stratify=pizza_not_dataset.targets)
    train_dataset = torch.utils.data.Subset(pizza_not_dataset, train_ind)
    test_dataset = torch.utils.data.Subset(pizza_not_dataset, test_ind)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

# h. parameters
    epoch = 1
    lr = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

# create train loop and test loop
    def train_loop(model, optimizer, criterion, train_loader):
        model.train()
        size, correct, running_loss = 0, 0, 0
        for i, data in enumerate(train_loader):
            size += len(data[1])
            X = data[0].to(device)
            y = data[1].unsqueeze(1)
            y = y.to(torch.float32)
            y = y.to(device)
            pred = model(X)
            optimizer.zero_grad()
            loss = criterion(pred, y)
            loss.backward()
            running_loss += loss.item() * len(y)
            optimizer.step()
            correct += (((pred > 0.5) == y).float().sum())
        accuracy = (correct / size) * 100
        print(f"train_loss/epoch: {running_loss / size}, accuracy/epoch: {accuracy}")

    def test_loop(model, criterion, test_loader):
        model.eval()
        size, correct = 0, 0
        with torch.no_grad():
            for i, data in enumerate(test_loader):
                size += len(data[1])
                X = data[0].to(device)
                y = data[1].unsqueeze(1)
                y = y.to(torch.float32)
                y = y.to(device)
                pred = model(X)
                loss = criterion(pred, y).item()
                correct += (((pred > 0.5) == y).float().sum())
                print(f"test_loss/batch: {loss}")
        accuracy = (correct / size) * 100
        print(f"accuracy: {accuracy}")

# train and test model
    for i in range(epoch):
        train_loop(model, optimizer, criterion, train_loader)
    test_loop(model, criterion, test_loader)
    torch.save(model.state_dict(), '/content/drive/MyDrive/DL_1/pizza_not_classifier/model.pth')
# show some predicted values
    show_predict(model, test_loader)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: ignored